# Differential RNA expression analysis

BI practicum 6

In this project we will study changes that happen in yeast cells before or during fermentation. We will explore how RNA expression levels change as yeast undergo fermentation to make bread rise. There are two replicates of RNA-seq data from yeast before and during fermentation, and our goal is to find out if the yeast express different genes during fermentation than they do under normal growth.



## 1. Data

**yeast reads:**

SRR941816: fermentation 0 minutes replicate 1 \
[ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941816/SRR941816.fastq.gz](http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941816/SRR941816.fastq.gz) \
SRR941817: fermentation 0 minutes replicate 2 \
[ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941817/SRR941817.fastq.gz](http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941817/SRR941817.fastq.gz) \
SRR941818: fermentation 30 minutes replicate 1 \
[ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941818/SRR941818.fastq.gz](http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941818/SRR941818.fastq.gz) \
SRR941819: fermentation 30 minutes replicate 2 \
[ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941819/SRR941819.fastq.gz](http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941819/SRR941819.fastq.gz)


As a reference genome we will use Saccharomyces cerevisiae, in the genome database at NCBI. Make sure you have strain S288c and assembly R64. Download the reference genome in FASTA format and annotation in GFF format.

reference genome file: 
[ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz](http://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz)

annotation file:
[ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.gff.gz](http://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.gff.gz)


## 2. Analysis Pipeline

There are plenty of different tools aiming to solve this task. Hereafter you can find a pipeline describing HISAT2 + deseq2, but you can try any aligner and DE count package of your choice. It’s worth checking out kallisto + sleuth (kallisto is based on an interesting algorithmic approach of pseudoalignment. And it’s blazingly fast.) 


For deseq2 you can find [here](https://figshare.com/articles/software/Scripts_for_RNA-seq_project/14239304) two scripts that can be useful for analysis (you also can try to write your own). 


### **a.** Aligning with HISAT2
build genome index:
- install hisat2
```
conda install bioconda::hisat2
```

- run hisat2-build
```
hisat2-build data/GCF_000146045.2_R64_genomic.fna data/GCF_000146045.2_R64_genomic.index
```

- run hisat2 in single-end mode:
```
hisat2 -p 5 -x data/GCF_000146045.2_R64_genomic.index -U data/SRR941816.fastq.gz | samtools sort > data/SRR941816.sorted.bam
hisat2 -p 5 -x data/GCF_000146045.2_R64_genomic.index -U data/SRR941817.fastq.gz | samtools sort > data/SRR941817.sorted.bam
hisat2 -p 5 -x data/GCF_000146045.2_R64_genomic.index -U data/SRR941818.fastq.gz | samtools sort > data/SRR941818.sorted.bam
hisat2 -p 5 -x data/GCF_000146045.2_R64_genomic.index -U data/SRR941819.fastq.gz | samtools sort > data/SRR941819.sorted.bam
```


### **b.** Quantifying with featureCounts


featureCounts can not work with GFF files. We need to convert the GFF file to GTF format. For this purpose we will use gffread. 

- install gffread:
```
conda install gffread
``` 

- convert from GFF to GTF:
```
gffread data/GCF_000146045.2_R64_genomic.gff -T -o data/GCF_000146045.2_R64_genomic.gtf
```

- run the feature counts program:
```
conda install bioconda::subread
```

```
featureCounts -g gene_id -a data/GCF_000146045.2_R64_genomic.gtf -o data/gene_counts.txt data/SRR941816.sorted.bam data/SRR941817.sorted.bam data/SRR941818.sorted.bam data/SRR941819.sorted.bam
```

Apparently, there are some rows where gene_id is missing and the file is not parsed correctly during the transformation. Consider removing those rows or use the original .gff with:
```
featureCounts -t gene -g ID -a data/GCF_000146045.2_R64_genomic.gff -o data/gene_counts.txt data/SRR941816.sorted.bam data/SRR941817.sorted.bam data/SRR941818.sorted.bam data/SRR941819.sorted.bam
```

We don’t need all columns from featureCounts output file for further analysis, so let’s simplify it.

- simplify the counts:
```
cat data/gene_counts.txt | cut -f 1,7-10 > data/simple_counts.txt
```

### **c.** Find differentially expressed genes with Deseq2

- calculate metrics:
```
cat data/simple_counts.txt | R -f src/deseq2.r
```

This script generates following files:
* result.txt will contain calculated metrics for our genes
* norm-matrix-deseq2.txt will contain normalised counts that we will use in visualisation

- draw heatmap:
```
cat norm-matrix-deseq2.txt | R -f src/draw-heatmap.r
```

Look at the output.pdf file.

## 3. Result Interpretation


In the result.txt file genes are sorted by adjusted p-values. Take the first 50 genes from this file (take the 1st column, take only the second element of splitted by '-' list):
```
head -n 50 result.txt | cut -f 1 | cut -d "-" -f 2 > genes.txt
```

There are too few down-regulated genes in this list so let's split it to top 50 up-regulated and top 50 down-regulated genes and enrich separately.
```
cat result.txt | sort -k4,4nr | head -n 50 | cut -f 1 | cut -d "-" -f 2 > top_50_upregulated.txt
cat result.txt | sort -k4,4n | head -n 50 | cut -f 1 | cut -d "-" -f 2 > top_50_downregulated.txt
```

Use gene ontology terms to get a sense of what these genes are doing

The [Saccharomyces Genome Database](http://www.yeastgenome.org/cgi-bin/GO/goSlimMapper.pl), maintained by Stanford, contains all of the GO terms associated with yeast.


### Results for top-50
<html><body>
<br><b>These 2 identifier(s) represent either invalid gene names, dubious ORFs unlikely to encode functional proteins or genes which are currently unannotated:</b> YER127W YLR264C<br><br><b>These 8 identifier(s) represent valid gene names that either could not be mapped to terms in the current GO slim set or are currently annotated to the root node for the slim set being used:</b> YGR079W YKL120W YLR327C YBL028C YOR360C YJL108C YJL107C YML018C<br><center>
<table border=0 cellpadding=2>
<tr align=center>
<td colspan=5><b>GO Terms from the biological process Ontology</b></td></tr>
<tr bgcolor=#993300>
<th align=center nowrap><font color=#FFFFFF>GO Term (GO ID)</font></th>
<th align=center><font color=#FFFFFF>Genes Annotated to the GO Term</font></th>
<th align=center><font color=#FFFFFF>GO Term Usage in Gene List</font></th>
<th align=center><font color=#FFFFFF>Genome Frequency of Use</font></th>
</tr>
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006364">rRNA processing</a> ( GO:0006364 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR449C">YDR449C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YEL026W">YEL026W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR159C">YGR159C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR066W">YHR066W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR196W">YHR196W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJL069C">YJL069C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YMR093W">YMR093W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL112W">YNL112W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL182C">YNL182C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL041C">YOL041C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL080C">YOL080C</a></td> <td align="left" nowrap="">11 of 47 genes, 23.40%</td> <td align="left" nowrap="">327 of 6479 annotated genes, 5.05%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042273">ribosomal large subunit biogenesis</a> ( GO:0042273 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YCR072C">YCR072C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDL063C">YDL063C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR066W">YHR066W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YIR012W">YIR012W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJL122W">YJL122W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL182C">YNL182C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL041C">YOL041C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL080C">YOL080C</a></td> <td align="left" nowrap="">8 of 47 genes, 17.02%</td> <td align="left" nowrap="">120 of 6479 annotated genes, 1.85%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042274">ribosomal small subunit biogenesis</a> ( GO:0042274 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR449C">YDR449C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YEL026W">YEL026W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR159C">YGR159C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR196W">YHR196W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJL069C">YJL069C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YMR093W">YMR093W</a></td> <td align="left" nowrap="">6 of 47 genes, 12.77%</td> <td align="left" nowrap="">135 of 6479 annotated genes, 2.08%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006360">transcription by RNA polymerase I</a> ( GO:0006360 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YHR196W">YHR196W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJL148W">YJL148W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJR063W">YJR063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YML043C">YML043C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YMR093W">YMR093W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL248C">YNL248C</a></td> <td align="left" nowrap="">6 of 47 genes, 12.77%</td> <td align="left" nowrap="">69 of 6479 annotated genes, 1.06%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042255">ribosome assembly</a> ( GO:0042255 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YCR072C">YCR072C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR159C">YGR159C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR066W">YHR066W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YIR012W">YIR012W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL182C">YNL182C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL080C">YOL080C</a></td> <td align="left" nowrap="">6 of 47 genes, 12.77%</td> <td align="left" nowrap="">74 of 6479 annotated genes, 1.14%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0055086">nucleobase-containing small molecule metabolic process</a> ( GO:0055086 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBL039C">YBL039C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YMR300C">YMR300C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL141W">YNL141W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL136C">YOL136C</a></td> <td align="left" nowrap="">4 of 47 genes, 8.51%</td> <td align="left" nowrap="">190 of 6479 annotated genes, 2.93%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0005975">carbohydrate metabolic process</a> ( GO:0005975 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBR105C">YBR105C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YER062C">YER062C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKR097W">YKR097W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL136C">YOL136C</a></td> <td align="left" nowrap="">4 of 47 genes, 8.51%</td> <td align="left" nowrap="">157 of 6479 annotated genes, 2.42%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006401">RNA catabolic process</a> ( GO:0006401 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR264W">YLR264W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL112W">YNL112W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR359W">YOR359W</a></td> <td align="left" nowrap="">3 of 47 genes, 6.38%</td> <td align="left" nowrap="">162 of 6479 annotated genes, 2.50%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0055085">transmembrane transport</a> ( GO:0055085 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR536W">YDR536W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR094C">YHR094C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL065W">YNL065W</a></td> <td align="left" nowrap="">3 of 47 genes, 6.38%</td> <td align="left" nowrap="">273 of 6479 annotated genes, 4.21%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0008643">carbohydrate transport</a> ( GO:0008643 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR536W">YDR536W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR094C">YHR094C</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">36 of 6479 annotated genes, 0.56%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006354">DNA-templated transcription elongation</a> ( GO:0006354 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YJL148W">YJL148W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL248C">YNL248C</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">105 of 6479 annotated genes, 1.62%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006353">DNA-templated transcription termination</a> ( GO:0006353 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YJR063W">YJR063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL112W">YNL112W</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">42 of 6479 annotated genes, 0.65%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0051052">regulation of DNA metabolic process</a> ( GO:0051052 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YNL182C">YNL182C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR359W">YOR359W</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">117 of 6479 annotated genes, 1.81%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006366">transcription by RNA polymerase II</a> ( GO:0006366 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YJR063W">YJR063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL112W">YNL112W</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">489 of 6479 annotated genes, 7.55%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0051603">proteolysis involved in protein catabolic process</a> ( GO:0051603 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBR105C">YBR105C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR224W">YLR224W</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">218 of 6479 annotated genes, 3.36%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0009451">RNA modification</a> ( GO:0009451 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YOL124C">YOL124C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YPL212C">YPL212C</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">175 of 6479 annotated genes, 2.70%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006629">lipid metabolic process</a> ( GO:0006629 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBL039C">YBL039C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL151W">YOL151W</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">312 of 6479 annotated genes, 4.82%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0008033">tRNA processing</a> ( GO:0008033 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YOL124C">YOL124C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YPL212C">YPL212C</a></td> <td align="left" nowrap="">2 of 47 genes, 4.26%</td> <td align="left" nowrap="">119 of 6479 annotated genes, 1.84%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0015931">nucleobase-containing compound transport</a> ( GO:0015931 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YHR196W">YHR196W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">122 of 6479 annotated genes, 1.88%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006970">response to osmotic stress</a> ( GO:0006970 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YER062C">YER062C</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">86 of 6479 annotated genes, 1.33%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006605">protein targeting</a> ( GO:0006605 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBR105C">YBR105C</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">216 of 6479 annotated genes, 3.33%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006397">mRNA processing</a> ( GO:0006397 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YEL026W">YEL026W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">170 of 6479 annotated genes, 2.62%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006260">DNA replication</a> ( GO:0006260 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YNL182C">YNL182C</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">138 of 6479 annotated genes, 2.13%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042221">response to chemical</a> ( GO:0042221 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YOR271C">YOR271C</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">469 of 6479 annotated genes, 7.24%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0070925">organelle assembly</a> ( GO:0070925 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR180W">YLR180W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">113 of 6479 annotated genes, 1.74%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0008380">RNA splicing</a> ( GO:0008380 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YEL026W">YEL026W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">139 of 6479 annotated genes, 2.15%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006418">tRNA aminoacylation for protein translation</a> ( GO:0006418 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR037W">YDR037W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">36 of 6479 annotated genes, 0.56%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0000054">ribosomal subunit export from nucleus</a> ( GO:0000054 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR264W">YLR264W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">67 of 6479 annotated genes, 1.03%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0033043">regulation of organelle organization</a> ( GO:0033043 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR180W">YLR180W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">236 of 6479 annotated genes, 3.64%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006520">amino acid metabolic process</a> ( GO:0006520 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR180W">YLR180W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">159 of 6479 annotated genes, 2.45%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006865">amino acid transport</a> ( GO:0006865 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YNL065W">YNL065W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">45 of 6479 annotated genes, 0.69%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006811">monoatomic ion transport</a> ( GO:0006811 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YNR060W">YNR060W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">97 of 6479 annotated genes, 1.50%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006091">generation of precursor metabolites and energy</a> ( GO:0006091 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YOL136C">YOL136C</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">82 of 6479 annotated genes, 1.27%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0002181">cytoplasmic translation</a> ( GO:0002181 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR264W">YLR264W</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">169 of 6479 annotated genes, 2.61%</td></tr> 
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0032787">monocarboxylic acid metabolic process</a> ( GO:0032787 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YOL136C">YOL136C</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">135 of 6479 annotated genes, 2.08%</td></tr> 
<tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006310">DNA recombination</a> ( GO:0006310 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGR159C">YGR159C</a></td> <td align="left" nowrap="">1 of 47 genes, 2.13%</td> <td align="left" nowrap="">193 of 6479 annotated genes, 2.98%</td></tr>
</table>
</center>

</body></html>

All top rows contain GO terms responsible for ribosomal interactions which seems to be logically justified based on what we expect from the fermenting yeast.

### Results for top-50 upregulated

<html><body>
<br><b>These 5 identifier(s) represent valid gene names that either could not be mapped to terms in the current GO slim set or are currently annotated to the root node for the slim set being used:</b> YCR021C YLR327C YJL153C YMR251W YBR298C<br><center>
<table border=0 cellpadding=2>
<tr align=center>
<td colspan=5><b>GO Terms from the biological process Ontology</b></td></tr>
<tr bgcolor=#993300>
<th align=center nowrap><font color=#FFFFFF>GO Term (GO ID)</font></th>
<th align=center><font color=#FFFFFF>Genes Annotated to the GO Term</font></th>
<th align=center><font color=#FFFFFF>GO Term Usage in Gene List</font></th>
<th align=center><font color=#FFFFFF>Genome Frequency of Use</font></th>
</tr>
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0005975">carbohydrate metabolic process</a> ( GO:0005975 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YAL038W">YAL038W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCR012W">YCR012W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDR050C">YDR050C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YER062C">YER062C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR192C">YGR192C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR254W">YGR254W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR174W">YHR174W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YIL053W">YIL053W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YIL162W">YIL162W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJL052W">YJL052W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJR009C">YJR009C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL060C">YKL060C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL152C">YKL152C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKR097W">YKR097W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR044C">YLR044C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL086C">YOL086C</a></td> <td align="left" nowrap="">16 of 50 genes, 32.00%</td> <td align="left" nowrap="">157 of 6479 annotated genes, 2.42%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006091">generation of precursor metabolites and energy</a> ( GO:0006091 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YAL038W">YAL038W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCR012W">YCR012W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDR050C">YDR050C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR192C">YGR192C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR254W">YGR254W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR174W">YHR174W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YIL057C">YIL057C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJL052W">YJL052W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJR009C">YJR009C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL060C">YKL060C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL152C">YKL152C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR044C">YLR044C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL086C">YOL086C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR374W">YOR374W</a></td> <td align="left" nowrap="">14 of 50 genes, 28.00%</td> <td align="left" nowrap="">82 of 6479 annotated genes, 1.27%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0055086">nucleobase-containing small molecule metabolic process</a> ( GO:0055086 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YAL038W">YAL038W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCR012W">YCR012W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDL022W">YDL022W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDR050C">YDR050C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR192C">YGR192C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR254W">YGR254W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR174W">YHR174W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJL052W">YJL052W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJR009C">YJR009C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL060C">YKL060C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL152C">YKL152C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL086C">YOL086C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR374W">YOR374W</a></td> <td align="left" nowrap="">13 of 50 genes, 26.00%</td> <td align="left" nowrap="">190 of 6479 annotated genes, 2.93%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0032787">monocarboxylic acid metabolic process</a> ( GO:0032787 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YAL038W">YAL038W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCR012W">YCR012W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDR050C">YDR050C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGL055W">YGL055W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR192C">YGR192C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR254W">YGR254W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR174W">YHR174W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJL052W">YJL052W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJR009C">YJR009C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL060C">YKL060C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL152C">YKL152C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR044C">YLR044C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR374W">YOR374W</a></td> <td align="left" nowrap="">13 of 50 genes, 26.00%</td> <td align="left" nowrap="">135 of 6479 annotated genes, 2.08%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0002181">cytoplasmic translation</a> ( GO:0002181 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YEL034W">YEL034W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGL103W">YGL103W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YKL056C">YKL056C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR029C">YLR029C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR075W">YLR075W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR340W">YLR340W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YML063W">YML063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL178W">YNL178W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YPL131W">YPL131W</a></td> <td align="left" nowrap="">9 of 50 genes, 18.00%</td> <td align="left" nowrap="">169 of 6479 annotated genes, 2.61%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042255">ribosome assembly</a> ( GO:0042255 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR075W">YLR075W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR167W">YLR167W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR340W">YLR340W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR063W">YOR063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR369C">YOR369C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YPL131W">YPL131W</a></td> <td align="left" nowrap="">6 of 50 genes, 12.00%</td> <td align="left" nowrap="">74 of 6479 annotated genes, 1.14%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006970">response to osmotic stress</a> ( GO:0006970 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDL022W">YDL022W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YER062C">YER062C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YFL014W">YFL014W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YIL053W">YIL053W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YPL240C">YPL240C</a></td> <td align="left" nowrap="">5 of 50 genes, 10.00%</td> <td align="left" nowrap="">86 of 6479 annotated genes, 1.33%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006364">rRNA processing</a> ( GO:0006364 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YJR123W">YJR123W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR167W">YLR167W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YML063W">YML063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR063W">YOR063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR369C">YOR369C</a></td> <td align="left" nowrap="">5 of 50 genes, 10.00%</td> <td align="left" nowrap="">327 of 6479 annotated genes, 5.05%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006457">protein folding</a> ( GO:0006457 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBR072W">YBR072W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR142W">YGR142W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLL026W">YLL026W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YPL240C">YPL240C</a></td> <td align="left" nowrap="">4 of 50 genes, 8.00%</td> <td align="left" nowrap="">99 of 6479 annotated genes, 1.53%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042273">ribosomal large subunit biogenesis</a> ( GO:0042273 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR075W">YLR075W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR340W">YLR340W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR063W">YOR063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YPL131W">YPL131W</a></td> <td align="left" nowrap="">4 of 50 genes, 8.00%</td> <td align="left" nowrap="">120 of 6479 annotated genes, 1.85%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042274">ribosomal small subunit biogenesis</a> ( GO:0042274 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YJR123W">YJR123W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR167W">YLR167W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YML063W">YML063W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR369C">YOR369C</a></td> <td align="left" nowrap="">4 of 50 genes, 8.00%</td> <td align="left" nowrap="">135 of 6479 annotated genes, 2.08%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006414">translational elongation</a> ( GO:0006414 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YEL034W">YEL034W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR249W">YLR249W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOR063W">YOR063W</a></td> <td align="left" nowrap="">3 of 50 genes, 6.00%</td> <td align="left" nowrap="">335 of 6479 annotated genes, 5.17%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0000054">ribosomal subunit export from nucleus</a> ( GO:0000054 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGL123W">YGL123W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YJR123W">YJR123W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YNL178W">YNL178W</a></td> <td align="left" nowrap="">3 of 50 genes, 6.00%</td> <td align="left" nowrap="">67 of 6479 annotated genes, 1.03%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0055085">transmembrane transport</a> ( GO:0055085 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDL022W">YDL022W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDR536W">YDR536W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGL008C">YGL008C</a></td> <td align="left" nowrap="">3 of 50 genes, 6.00%</td> <td align="left" nowrap="">273 of 6479 annotated genes, 4.21%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042221">response to chemical</a> ( GO:0042221 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDL022W">YDL022W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YFL014W">YFL014W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR110C">YLR110C</a></td> <td align="left" nowrap="">3 of 50 genes, 6.00%</td> <td align="left" nowrap="">469 of 6479 annotated genes, 7.24%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0033043">regulation of organelle organization</a> ( GO:0033043 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGR254W">YGR254W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR174W">YHR174W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YPL240C">YPL240C</a></td> <td align="left" nowrap="">3 of 50 genes, 6.00%</td> <td align="left" nowrap="">236 of 6479 annotated genes, 3.64%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0009408">response to heat</a> ( GO:0009408 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBR072W">YBR072W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YFL014W">YFL014W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLL026W">YLL026W</a></td> <td align="left" nowrap="">3 of 50 genes, 6.00%</td> <td align="left" nowrap="">76 of 6479 annotated genes, 1.17%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0048284">organelle fusion</a> ( GO:0048284 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGR254W">YGR254W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR174W">YHR174W</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">99 of 6479 annotated genes, 1.53%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006811">monoatomic ion transport</a> ( GO:0006811 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGL008C">YGL008C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGR192C">YGR192C</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">97 of 6479 annotated genes, 1.50%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0007005">mitochondrion organization</a> ( GO:0007005 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR077W">YDR077W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGL055W">YGL055W</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">253 of 6479 annotated genes, 3.90%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0071554">cell wall organization or biogenesis</a> ( GO:0071554 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR077W">YDR077W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLR110C">YLR110C</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">195 of 6479 annotated genes, 3.01%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0009311">oligosaccharide metabolic process</a> ( GO:0009311 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YIL162W">YIL162W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YLL026W">YLL026W</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">43 of 6479 annotated genes, 0.66%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006629">lipid metabolic process</a> ( GO:0006629 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGL055W">YGL055W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL151W">YOL151W</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">312 of 6479 annotated genes, 4.82%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006520">amino acid metabolic process</a> ( GO:0006520 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YLR044C">YLR044C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YOL086C">YOL086C</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">159 of 6479 annotated genes, 2.45%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0007033">vacuole organization</a> ( GO:0007033 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGR254W">YGR254W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YHR174W">YHR174W</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">103 of 6479 annotated genes, 1.59%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006417">regulation of translation</a> ( GO:0006417 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YEL034W">YEL034W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YGL123W">YGL123W</a></td> <td align="left" nowrap="">2 of 50 genes, 4.00%</td> <td align="left" nowrap="">151 of 6479 annotated genes, 2.33%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006605">protein targeting</a> ( GO:0006605 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YPL240C">YPL240C</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">216 of 6479 annotated genes, 3.33%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006401">RNA catabolic process</a> ( GO:0006401 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YNL178W">YNL178W</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">162 of 6479 annotated genes, 2.50%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0032200">telomere organization</a> ( GO:0032200 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YPL240C">YPL240C</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">135 of 6479 annotated genes, 2.08%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0016197">endosomal transport</a> ( GO:0016197 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGR142W">YGR142W</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">87 of 6479 annotated genes, 1.34%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0008643">carbohydrate transport</a> ( GO:0008643 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR536W">YDR536W</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">36 of 6479 annotated genes, 0.56%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006865">amino acid transport</a> ( GO:0006865 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGR142W">YGR142W</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">45 of 6479 annotated genes, 0.69%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0007031">peroxisome organization</a> ( GO:0007031 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDL022W">YDL022W</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">55 of 6479 annotated genes, 0.85%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0051052">regulation of DNA metabolic process</a> ( GO:0051052 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YPL240C">YPL240C</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">117 of 6479 annotated genes, 1.81%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0070925">organelle assembly</a> ( GO:0070925 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGL008C">YGL008C</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">113 of 6479 annotated genes, 1.74%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006413">translational initiation</a> ( GO:0006413 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YEL034W">YEL034W</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">68 of 6479 annotated genes, 1.05%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006979">response to oxidative stress</a> ( GO:0006979 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YFL014W">YFL014W</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">99 of 6479 annotated genes, 1.53%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0048308">organelle inheritance</a> ( GO:0048308 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YGL055W">YGL055W</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">55 of 6479 annotated genes, 0.85%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0051604">protein maturation</a> ( GO:0051604 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YPL240C">YPL240C</a></td> <td align="left" nowrap="">1 of 50 genes, 2.00%</td> <td align="left" nowrap="">82 of 6479 annotated genes, 1.27%</td></tr></table>
</center>

</body></html>

Upregulation seems very logic too. `carbohydrate metabolic process` and `cytoplasmic translation` are upregulated indeed.

### Results for top-50 downregulated

<html><body>
<br><b>These 7 identifier(s) represent either invalid gene names, dubious ORFs unlikely to encode functional proteins or genes which are currently unannotated:</b> YAL067W YBL100W YBR012W YCL058W YDR210C YDR261W YDR365W<br><br><b>These 10 identifier(s) represent valid gene names that either could not be mapped to terms in the current GO slim set or are currently annotated to the root node for the slim set being used:</b> Q0060 YDR210W YAL064W YDR316W YBL112C YCL068C YCR108C YCR041W YAL063C Q0075<br><center>
<table border=0 cellpadding=2>
<tr align=center>
<td colspan=5><b>GO Terms from the biological process Ontology</b></td></tr>
<tr bgcolor=#993300>
<th align=center nowrap><font color=#FFFFFF>GO Term (GO ID)</font></th>
<th align=center><font color=#FFFFFF>Genes Annotated to the GO Term</font></th>
<th align=center><font color=#FFFFFF>GO Term Usage in Gene List</font></th>
<th align=center><font color=#FFFFFF>Genome Frequency of Use</font></th>
</tr>
<tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006366">transcription by RNA polymerase II</a> ( GO:0006366 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBL005W">YBL005W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCL066W">YCL066W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCL067C">YCL067C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCR039C">YCR039C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCR040W">YCR040W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDR034C">YDR034C</a></td> <td align="left" nowrap="">6 of 37 genes, 16.22%</td> <td align="left" nowrap="">489 of 6479 annotated genes, 7.55%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006310">DNA recombination</a> ( GO:0006310 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/Q0050">Q0050</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0055">Q0055</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0065">Q0065</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0070">Q0070</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0160">Q0160</a></td> <td align="left" nowrap="">5 of 37 genes, 13.51%</td> <td align="left" nowrap="">193 of 6479 annotated genes, 2.98%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0008380">RNA splicing</a> ( GO:0008380 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/Q0065">Q0065</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0110">Q0110</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0115">Q0115</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0120">Q0120</a></td> <td align="left" nowrap="">4 of 37 genes, 10.81%</td> <td align="left" nowrap="">139 of 6479 annotated genes, 2.15%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006397">mRNA processing</a> ( GO:0006397 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/Q0050">Q0050</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0115">Q0115</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/Q0120">Q0120</a></td> <td align="left" nowrap="">3 of 37 genes, 8.11%</td> <td align="left" nowrap="">170 of 6479 annotated genes, 2.62%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0032196">transposition</a> ( GO:0032196 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YAR009C">YAR009C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YAR010C">YAR010C</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YCL020W">YCL020W</a></td> <td align="left" nowrap="">3 of 37 genes, 8.11%</td> <td align="left" nowrap="">100 of 6479 annotated genes, 1.54%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0042221">response to chemical</a> ( GO:0042221 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBL005W">YBL005W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YBR020W">YBR020W</a></td> <td align="left" nowrap="">2 of 37 genes, 5.41%</td> <td align="left" nowrap="">469 of 6479 annotated genes, 7.24%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0005975">carbohydrate metabolic process</a> ( GO:0005975 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBR020W">YBR020W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDL246C">YDL246C</a></td> <td align="left" nowrap="">2 of 37 genes, 5.41%</td> <td align="left" nowrap="">157 of 6479 annotated genes, 2.42%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0055085">transmembrane transport</a> ( GO:0055085 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YCL069W">YCL069W</a>, <a target="infowin" href= "https://www.yeastgenome.org/locus/YDR039C">YDR039C</a></td> <td align="left" nowrap="">2 of 37 genes, 5.41%</td> <td align="left" nowrap="">273 of 6479 annotated genes, 4.21%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006897">endocytosis</a> ( GO:0006897 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YBR266C">YBR266C</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">103 of 6479 annotated genes, 1.59%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006873">intracellular monoatomic ion homeostasis</a> ( GO:0006873 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR098C">YDR098C</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">132 of 6479 annotated genes, 2.04%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006520">amino acid metabolic process</a> ( GO:0006520 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR034C">YDR034C</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">159 of 6479 annotated genes, 2.45%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0008033">tRNA processing</a> ( GO:0008033 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/RPM1">RPM1</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">119 of 6479 annotated genes, 1.84%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006811">monoatomic ion transport</a> ( GO:0006811 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR039C">YDR039C</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">97 of 6479 annotated genes, 1.50%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0006865">amino acid transport</a> ( GO:0006865 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YCL069W">YCL069W</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">45 of 6479 annotated genes, 0.69%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0071554">cell wall organization or biogenesis</a> ( GO:0071554 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR261C">YDR261C</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">195 of 6479 annotated genes, 3.01%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0007010">cytoskeleton organization</a> ( GO:0007010 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/YDR098C">YDR098C</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">252 of 6479 annotated genes, 3.89%</td></tr> <tr style="color:black;", bgcolor="FFE4C4"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0032543">mitochondrial translation</a> ( GO:0032543 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/Q0140">Q0140</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">170 of 6479 annotated genes, 2.62%</td></tr> <tr style="color:black;", bgcolor="DEB887"><td align="left" nowrap=""><a target="infowin" href="https://www.yeastgenome.org/go/GO:0055086">nucleobase-containing small molecule metabolic process</a> ( GO:0055086 )</td> <td align="left"><a target="infowin" href= "https://www.yeastgenome.org/locus/Q0080">Q0080</a></td> <td align="left" nowrap="">1 of 37 genes, 2.70%</td> <td align="left" nowrap="">190 of 6479 annotated genes, 2.93%</td></tr></table>
</center>

</body></html>

When the translation starts transcription is comming to its end so this seems legit too.